# 3.13.31 APIs, Scripts & the Operating System

### Another API call

Let's look at one more API call using the [CoinGecko API](https://www.coingecko.com/en/api/documentation). Specifically, we'll use the `/coins/{id}/history` endpoint to get the historical exchange rates for bitcoin at a given date. To do this, we replace `{id}` with bitcoin and add the `date` parameter as follows: 

    https://api.coingecko.com/api/v3/coins/bitcoin/history?date=01-06-2022

In [ ]:
import pandas as pd
import requests

In [ ]:
url = 'https://api.coingecko.com/api/v3/coins/bitcoin/history?date=01-06-2022'
r = requests.get(url)
print(r.status_code)   # check the status code
df = pd.DataFrame.from_dict(r.json()['market_data'])
print(df.shape)
df.head()

The result is a 61x3 DataFrame, which includes the `current_price`, the `market_cap` and the `total_volume` of transactions between bitcoin and each of the 61 available tickers on the 1st of June, 2022. 

If we wanted to **save to a .csv file** this DataFrame, we can use the `.to_csv()` method, specifying the filename: 

In [ ]:
df.to_csv('df_test.csv')

### Scripts

If we wanted to **build a historical account** of these metrics, say the last 28 days, we would have to make several API calls, each with a different `date` parameter and then append them together in a final DataFrame. 

Such a task could be performed here on a Jupyter Notebook, but this is actually a great time to introduce another way of writing Python code, that is, **using a script**. 

<img src="img/script-notebook.png" width="600">

Scripts are not unique to Python, actually most languages use a script to execute their code. The main difference between the two is that a **Jupyter Notebook allows you to add titles, text with commentary, pictures and it renders charts and outputs** in-line with your code, while **scripts are simple text files** (with a `.py` extension) that contain just code (and comments to the code) and **can execute the whole set of instructions at once**. 

We will use **Spyder**, a Python scripting IDE that comes **pre-installed with the Anaconda distribution**. Although it doesn't have as many bells and whistles as some of its competition (like *PyCharm* or *Visual Studio Code*), its simplicity is well suited for a first approach and its features include everything we need for our analytical purpouses. 

You can start up Spyder by typing `spyder` in: 
- a Conda terminal (if you're using a Windows OS)
- any terminal (if you're using a Mac or Linux OS)

Let's move to the `script1_btc_28_days.py` script to see how we can implement a script that performs the actions described above. The algorithm, broadly speaking, will look like this: 

- create a list of dates that ranges from 28 days ago to today
- initialise an empty DataFrame named `df`
- loop through the list of dates and, at each cycle: 
    - make an HTTP request to the CoinGecko API 
    - retrieve the information we need in JSON format
    - save the data to a temporary DataFrame named "tmp_df"
    - add the current cycle's date to a new column name "date" in the DataFrame
    - append the temporary DataFrame to the "df" DataFrame
- analyse or plot the final DataFrame

### Interacting with the OS

Python's Standard Library includes a very useful utility package, called `os`, that **allows you to interact with the Operating System** and move between folders, create and delete files and perform all those actions that you would usually do via the computer's graphical user interface. 

In [ ]:
import os

A good starting point would be to know your **current working directory**, that is, the location in your file system where you are executing this file from (by default, that is where the file has been saved). You can retrieve this information using he `os.getcwd()` function: 

In [ ]:
# show the current working directory (cwd)
os.getcwd()

If you want to **move to another folder**, you can use the `os.chdir()` function, where you can specify the new directory's path:

In [ ]:
# change the current directory
os.chdir('/Users/franz/Documents/Boolean/Data Analytics/git')

Another useful function is `os.listdir()`, which simply **lists all the files and folders** that are present in your current working directory: 

In [ ]:
# list all the files and directories in the cwd
os.listdir()

The functions `os.mkdir()` and `os.rmdir` allow you to, respectively, **create and remove a folder** from the current working directory:

In [ ]:
# move to this class' folder
cwd = '/Users/franz/Documents/Boolean/Data Analytics/git/3.13.32 APIs, Scripts and OS'
os.chdir(cwd)

# create a new directory called 'test'
os.mkdir('test')

# check that the directory is effectively there
os.listdir()

In order to **move a file or a folder** (the method is the same), you simply **rename** the file or folder, changing its full path. Let's see an example: let's create another folder "test2" and move it inside the folder "test" we just created:

In [ ]:
# create a new directory called 'test2'
os.mkdir('test2')

# move the directory
os.rename(cwd+'/test2', cwd+'/test/test2')

Let's remove "test2" and then "test" *(note: if a folder is not empty you won't be able to remove it with this command, check out [this web page](https://www.codegrepper.com/code-examples/python/delete+non+empty+directory+python) for more information on how to do that)*:

In [ ]:
# remove the directory
os.rmdir('test/test2')
os.rmdir('test')

# check that the directory is no longer there
os.listdir()

Finally, to remove a file, you can use the `os.remove()` function: 

In [ ]:
os.remove('df_test.csv')

### A final example

Let's use what we just learned in a practical context; we want to write two more scripts: 

- in the first script we retrieve bitcoin's prices (just like we did for the previous script) but this time we're going to save each temporary DataFrame we're creating at each cycle of the loop to a local .csv file in the `/data/btc` directory;
- in the second script, we access the `/data/btc` directory, list all of its contents (that is, the files we just saved) and, looping through each one of them, we load them to a temporary DataFrame and then append each file to an (initially) empty DataFrame; we finally print a linechart of the data and save a .png copy to the local folder. 